In [1]:
from bs4 import BeautifulSoup
import pandas as pd
import requests
import re

In [2]:
url = requests.get('https://www.westmont.edu/academics/majors-programs')
soup = BeautifulSoup(url.text)

In [3]:
majors = soup.find('div',{'class':['field field--name-body','field--type-text-with-summary','field--label-hidden field--item']})
major_dic = dict()

templete = 'https://www.westmont.edu'
for a in majors.find_all('a'):
    
    reg = re.search(r"(.*)\s\(.*|.*",a.text)
    major_title = reg.group(1) if reg.group(1) else reg.group(0)
    is_http = re.match(r'https:',a['href'])
    if is_http:
        major_dic[major_title] = a['href']
    else:
        major_dic[major_title] = templete + a['href']

In [4]:
soup_dic = {major: BeautifulSoup(requests.get(url).text) for major, url in major_dic.items()}    

In [76]:
scrap_dic = dict.fromkeys(soup_dic.keys(),[])
for major, soup in soup_dic.items():
    first = soup.select('div.field--type-text-with-summary')
    
    string = ''
    for p in first[0].find_all('p'):
        string += p.text
    
    scrap_dic[major] = list()
    scrap_dic[major].append(string)

In [77]:
df = pd.DataFrame.from_dict(data=scrap_dic,orient='index')

In [87]:
df_filtered = df.drop(df.index[-13:],axis=0)

In [88]:
df_filtered.to_json('Westmont_major.json')